# QGraphs Data - Versions

*This collection of notebooks explores creation and use of* ***QGgraphs (Quantum Digraphs)*** *in* ***Networkx+PennyLane+PyTorch**.*

**By:** Jacob Cybulski<br>
**Date:** August 2024<br>
**Aims:** The goal of this notebook is to test some *Quantum Digraph (QGraphs)* features and functions.<br/>
**Refs:**
- Brownlee, J., 2019. <a href="https://machinelearningmastery.com/divergence-between-probability-distributions/" target="_blank">How to Calculate the KL Divergence for Machine Learning</a>. MachineLearningMastery.com.
- <a href="https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence" target="_blank">Kullback–Leibler divergence</a>, Wikipedia.
- <a href="https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence" target="_blank">Jensen-Shannon Divergence</a>, Wikipedia.
- <a href="https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html" target="_blank">KLDivLoss</a>. PyTorch docs.

**PennyLane technical refs:**
- https://pennylane.ai/codebook/
- https://docs.pennylane.ai/en/stable/index.html
- https://docs.pennylane.ai/en/stable/introduction/interfaces.html (see refs to Fourier and fun fitting examples below)
- https://docs.pennylane.ai/en/stable/code/api/pennylane.qnn.TorchLayer.html (see ref to an example below)
- https://pytorch.org/docs/stable/optim.html
- Example of circuit building: https://pennylane.ai/qml/demos/tutorial_learning_few_data/
- Example of circuit building: https://pennylane.ai/qml/demos/tutorial_expressivity_fourier_series/
- Example of training: https://pennylane.ai/qml/demos/function_fitting_qsp/
- Example of creating Torch nn layer: https://pennylane.ai/qml/demos/tutorial_qnn_module_torch/

**PyTorch technical refs:**
- Initialise weights: https://stackoverflow.com/questions/49433936/how-do-i-initialize-weights-in-pytorch
- First NN in PyTorch: https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/
- PyTorch training loop: https://machinelearningmastery.com/creating-a-training-loop-for-pytorch-models/

## What are QGraphs

A *QGraph* is a weighted directed graph, which excludes parallel edges (in the same direction), but allowing loops. It defines a stochastic navigation between its verteces, where the probability of taking a particular edge from a given vertex is given by the edge weight.

Formally, a qgraph $G$ is defined as a weighted digraph:

$$G = \{V, E, w\},$$

where $V = \{v\}$ is a set of digraph vertices, $E = \{(v_s,v_t) : v_s, v_t \in V\}$ is a set of directional edges, and $|V|$ being the number of vertices in $V$.<br/>
Function $w(V, V)\rightarrow R$ assigns weights to all edges, such that:

$$w(v_s, v_t) =
\left\{
	\begin{array}{ll}
		p(v_t | v_s)  & \forall_{v_s, v_t \in V,\;(v_s, v_t) \in E}\\
		0 & \forall_{v_s, v_t \in V,\;(v_s, v_t) \notin E}
	\end{array}
\right.
$$

where $p(v_t | v_s)$ is the probability of navigating over the edge $(v_s, v_t)$ when arrived at the source vertex $v_s$ to its target $v_t$ (including the self), and:

$$\forall_{(v_s, v_t) \in E}\: \sum_{v_t} w(v_s, v_t) = 1.$$

This means that every digraph vertex must have at least one outgoing link.

An arbitrary weighted digraph $G = \{V, E, w(V, V)\}$ can be expanded to become a QGraph $G^{'} = \{V^{'}, E^{'}, w^{'}(V^{'}, V^{'})\}$ by adding loops to all verteces $v$ with outdegree 0, and rescaling weights to add up to 1. Now:

$$\forall_{v \in V}\; v \in V^{'} \land \forall_{e \in E}\; e \in E^{'}$$
$$\forall_{v_s \in V}\;(\forall_{v_t \in V}\;(v_s, v_t) \notin E) \Longrightarrow (v_s, v_s) \in E^{'} \land w^{'}(v_s, v_s) = 1$$
$$\forall_{(v_s, v_t) \in E}\; w^{'}(v_s, v_t) = \frac{w(v_s, v_t)}{\sum_{(v_s, v_i) \in E}\; w(v_s, v_i)}$$

## PennyLane Implementation

A QGraph $G = \{V, E, w\}$, with $v \in V$ as the vertex unique numeric identifier, represents a parameterised function $f_\theta(V) \rightarrow V$,
which can be realised as a quantum circuit $U(v, \theta)$ measured projectively with observable $\mathcal{M}$:

$$
\begin{gather}
    f_\theta(v) = tr[ \mathcal{M} U(v, \theta)^\dagger \vert 0 \rangle \langle 0 \vert U(v, \theta) ],\quad and\\
    U(v, \theta) = A(\theta)S(v).
\end{gather}
$$

$S(v)$ is a feature map and $A(\theta)$ an ansatz with trainable parameters $\theta = (\theta_0, \theta_1, ..., \theta_n)$.
The ansatz can be trained on the graph edges $e = (v_s, v_t) \in E$ and their weights $w(v_s, v_t)$, such that $v_s$ would be passed as the input to $U(v, \theta)$, which is subsequently executed repeatedly to produce a distribution of possible outcomes $v_t$, each approximating the probability $p(v_t|v_s)$ aimed to approach the value of its edge weight $w(v_s, v_t)$.

The circuit $U(v, \theta)$ training can be facilitated by running $U(v_s, \theta)$ repeatedly for each vertex $v_s$ and measuring the difference between two probability distributions of possible paths from vertex $v_s$, i.e. the observed probability distribution $P(v_s) = \{p(v_t | v_s) : \forall_{v_t \in V}\}$ and the expected distribution $W(v_s) = \{w(v_s, v_t) : \forall_{e=(v_s, v_t) \in E}\}$. 
The standard loss function that could support the model training is $L1$ or $L2$, which would measure the distance between pairs of probability distributions $W(v_s)$ and $P(v_s)$. As the size of $P(v_s)$ distribution cannot be predicted in advance, hence the computational complexity of the $L1(W(v_s), P(v_s))$ measurement would be $O({\vert V \vert}^2)$.
The entire cost would then be measured as $MAE(W, P)$ or $MSE(W, P)$ with the complexity of $O({\vert V \vert}^3)$.

An alternative loss function is *Kullback-Leibler Divergence*, stated as a function $KL(W(v_s) || P(v_s))$ representing the magnitude of our "surprise" when the model adopts the observed distribution $P(v_s)$ instead of the actual distribution $W(v_s)$. The $KL$ loss function for the vertex $v_s$ can be defined as follows (adapted from Brownlee 2019):

$$KL(W(v_s) || P(v_s)) = \sum_{(v_s, v_t) \in E} w(v_s, v_t) * log(w(v_s, v_t) / p(v_t | v_s))$$

Therefore, the cost (average loss) of differences between observed and expected navigations for all digraph vertices is:

$$
\begin{array}{ll}
    KL(W || P) 
    &=&\frac{1}{|V|}\sum_{v_s \in V}\;KL(W(v_s) || P(v_s))\\
    &=&\frac{1}{|V|}\sum_{v_s \in V}\sum_{(v_s, v_t) \in E}\; w(v_s, v_t) * log(w(v_s, v_t) / p(v_t | v_s)) \\
    &=&\frac{1}{|V|}\sum_{(v_s, v_t) \in E}\; w(v_s, v_t) * log(w(v_s, v_t) / p(v_t | v_s))
\end{array}
$$

For the $KL$ function to be computable, we only consider pairs of vertices $(v_s, v_t)$ that have non-zero weights $w(v_s, v_t)$ and non-zero probability $p(v_t | v_s)$. 
Also note that $KL$ ignores those vertex pairs which emerge as "probably navigable" during model training, but not actually connected by an edge, meaning their pairwise weight is zero. 
This leads to the more selective cost calculation, with its complexity to be $O(|V||E|)$, with the upper bound of $O(|V|^3)$, however significantly lower for sparse graphs.

*Kullback-Leibler Divergence* is not symmetrical, i.e. $KL(W || P) \neq KL(P || W)$, and tends to generate very large negative values. Therefore, a related *Jensen-Shannon Divergence* could be used as a replacement. $JS$ loss function can be defined as follows (adapted from Brownlee 2019):

$$
\begin{array}{ll}
    &&\text{the loss function is:}\\
    \\
    &&for\; M(v_s) = 0.5\; (W(v_s) + P(v_s)),\\
    JS(W(v_s) || P(v_s)) 
    &=&0.5 * KL(W(v_s) || M(v_s)) + 0.5 * KL(P(v_s) || M(v_s)),\\
    \\
    &&\text{and the cost (average loss) is:}\\
    \\
    &&for\; M = 0.5\; (W + P),\\
    JS(W || P) 
    &=&\frac{1}{|V|}\sum_{v_s \in V}\;JS(W(v_s) || P(v_s))\\
    &=&0.5 * \frac{1}{|V|}\sum_{v_s \in V}\;(KL(W(v_s) || M(v_s)) + KL(P(v_s) || M(v_s))) \\
    &=&0.5 * (\frac{1}{|V|}\sum_{v_s \in V}\;KL(W(v_s) || M(v_s)) + \frac{1}{|V|}\sum_{v_s \in V}\;KL(P(v_s) || M(v_s))) \\
    &=&0.5 * (KL(W || M) + KL(P || M))
\end{array}
$$

As noted by Brownlee (2019), when compared with KL divergence, $JS(W(v_s) || P(v_s))$, and consequently $JS(W || P)$, is a more useful measure for model training - it is smooth and normalized, and, when using the base-2 logarithm, its scores are in the range from 0 (identical) to 1 (maximally different). Its computational complexity is also $O(|V||E|)$.

Note that in PyTorch, the class <a href="https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html" target="_blank">KLDivLoss</a> (see examples) defines the KL function, which takes as its arguments *input* (P), which is the result produced by a model (e.g. NN), and a *target* (W) which defines observations drawn from the data.

## Versions
All notebooks will keep consistent version numbers

*pl_qgraphs_main:* Commenced work on *QGraphs (QG)*
- **V1_00 (240808)** Initial graph representation in *networkx*<br/>
  Added an extra vertex for redirection from stop vertices
- **V1_01 (240814)** Added mathematical formulation
- **V1_02 (240816)** Changed representation and maths<br/>
  Added loops for redirection from stop vertices
- **V1_03 (240824)** Initial work on graph training in *PennyLane*
- **V1_04 (240828)** Added KL and JSD loss/cost functions
- **V1_05 (240901)** Notebook split into data and training parts<br/>
  As the code was getting very long, it was split into data prep and model training
- **V1_14 (241228)** Created main versioned after training component<br/>
  All main description moved into this "main" notebook

*pl_qgraphs_data:* Digraph generation
- **V1_05 (240901)** Generation of an 8 vertices graphs
- **V1_06 (240909)** Generation of a 16 vertices graphs
- **V1_07 (240909)** Change of the log file naming conventions
- **V1_10 (240911)** utils.Digraph.py plots save eps image, logs are created to save data
- **V1_14 (241228)** Cleanup

*pl_qgraphs_train:* QGraph model creation and training
- **V1_05 (240901)** Added QG model creation, started training
- **V1_06 (240908)** Need to recode QG input, from a single n to its binary representation
- **V1_07 (240909)** Fixed classical and hybrid modes, improved performance, esp. with JSD
- **V1_08 (240909)** Larger graph tested, code slightly reorganised
- **V1_09 (240909)** Testing different modes: quantum, classic and hybrid
- **V1_10 (240911)** Fixed classic model with JSD by adding softmax
- **V1_11 (240912)** Included side-by-side comparison of the expected vs observed weight distributions
- **V1_12 (241219)** Added several performance scores
- **V1_13 (241219)** Logs are incorporated to save graph definitions and performance data
- **V1_14 (241228)** QGraphs are now being consistently normalised + training made independent from analysis notebook

*pl_qgraphs_analysis:* QGraph model analysis
- **V1_14 (241228)** Analysis made independent from training notebook

## System

In [1]:
import sys
print(f'Environment:\n\n{sys.prefix}\n')

Environment:

/home/jacob/miniconda3/envs/pl-cuda12



In [2]:
!pip list | grep -e torch -e PennyLane -e networkx

networkx                  3.4.2
PennyLane                 0.39.0
PennyLane_Lightning       0.39.0
PennyLane_Lightning_GPU   0.39.0
PennyLane-Rigetti         0.39.0
torch                     2.5.1
torch_geometric           2.5.3
torchaudio                2.5.1
torcheval                 0.0.7
torchsummary              1.5.1
torchvision               0.20.1
